# Data Preprocessing

## Outlier Removal Before Filtering
Why: Removing outliers before bandpass filtering prevents filter artifacts and ensures the filtering process operates on clean data.
Method: The remove_outliers function detects outliers using Z-score thresholding and replaces them with interpolated values.

## Zero-Phase Filtering
Implementation: Replaced lfilter with filtfilt in the butter_bandpass_filter function.
Benefit: Zero-phase filtering avoids phase distortions that can be introduced by standard filtering methods, preserving the temporal characteristics of the EEG signals.

## Differential Entropy Computation
Adjustment: Added a small constant (1e-6) to the variance in compute_DE to prevent mathematical errors due to zero variance.
Importance: Ensures that DE values are computed accurately without encountering logarithm of zero.

## Data Reshaping to 10-20 System
Purpose: Aligns the processed data with the standard 10-20 EEG electrode placement system.
Manual Mapping: Channels are manually assigned to positions on an 8x9 grid. This requires careful attention to ensure accurate representation.

## Data Segregation
Data Handling:

- Processes all 24 trials for multiple participants, stacking segment-level DE features and corresponding labels into a unified dataset.
Labels are assigned at the segment level based on predefined trial labels.

Output Files:

- Saves the computed features and labels to .npy files:
X_1D.npy and y.npy: Containing raw DE features (62 channels × 5 bands) and labels.
X89.npy: Containing the reshaped 8×9×5 DE features aligned with the 10-20 system.

In [2]:
##############
## Extract DE features for 5 frequency bands from each channel of the SEED dataset,
## and convert the 62-channel data into an 8*9*5 three-dimensional input,
## where 8*9 represents the 2D plane after converting the 62 channels, and 5 represents the 5 frequency bands
##############

import os
import sys
import math
import numpy as np
import scipy.io as sio
from sklearn import preprocessing
from scipy.signal import butter, lfilter
from scipy.io import loadmat


# Function to remove outliers from a signal
def remove_outliers(signal, method='zscore', threshold=3):
    if method == 'zscore':
        mean = np.mean(signal)
        std = np.std(signal)
        z_scores = np.abs((signal - mean) / std)
        mask = z_scores < threshold
        signal_clean = signal.copy()
        # Replace outliers with interpolated values
        indices = np.arange(len(signal))
        signal_clean[~mask] = np.interp(indices[~mask], indices[mask], signal_clean[mask])
    elif method == 'mad':
        median = np.median(signal)
        mad = np.median(np.abs(signal - median))
        mask = np.abs(signal - median) < threshold * mad
        signal_clean = signal.copy()
        indices = np.arange(len(signal))
        signal_clean[~mask] = np.interp(indices[~mask], indices[mask], signal_clean[mask])
    else:
        raise ValueError("Method not recognized.")
    return signal_clean


# Function to create bandpass filter coefficients
def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs  # Nyquist frequency
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

# Function to apply bandpass filter to data
def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

# Function to compute Differential Entropy (DE) of a signal
def compute_DE(signal):
    variance = np.var(signal, ddof=1) + 1e-6  # Add a small epsilon to prevent log(0)
    return math.log(2 * math.pi * math.e * variance) / 2

# Function to decompose EEG data into different frequency bands and extract DE features
# Input: file path, name (shortened name of the participant)
def decompose(file, name):
    # Load the .mat file containing the EEG data
    data = loadmat(file)
    frequency = 200  # Sampling rate of the SEED dataset is downsampled to 200Hz

    # Label for each of the 24 trials
    all_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3,
                 2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1,
                 1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]

    # Create empty arrays to store DE features and labels
    decomposed_de = np.empty([0, 62, 5])
    label = np.array([])

    # Loop through all 24 trials in the dataset
    for trial in range(24):
        # Load the EEG data for the current trial
        tmp_trial_signal = data[name + '_eeg' + str(trial + 1)]

        # Number of samples per segment (0.5 seconds per segment, with a sampling rate of 200Hz)
        num_sample = int(len(tmp_trial_signal[0]) / 100)
        print('{}-{}'.format(trial + 1, num_sample))

        # Initialize temporary array to store DE features for each channel
        temp_de = np.empty([0, num_sample])
        # Assign labels for each sample in the current trial
        label = np.append(label, [all_label[trial]] * num_sample)

        # Loop through each channel (total 62 channels)
        for channel in range(62):
            trial_signal = tmp_trial_signal[channel]
            
            # Remove outliers before filtering
            trial_signal = remove_outliers(trial_signal, method='zscore', threshold=3)
            
            # Apply bandpass filters to extract different frequency bands
            delta = butter_bandpass_filter(trial_signal, 1, 4, frequency, order=3)
            theta = butter_bandpass_filter(trial_signal, 4, 8, frequency, order=3)
            alpha = butter_bandpass_filter(trial_signal, 8, 14, frequency, order=3)
            beta = butter_bandpass_filter(trial_signal, 14, 31, frequency, order=3)
            gamma = butter_bandpass_filter(trial_signal, 31, 51, frequency, order=3)

            # Initialize arrays to store DE values for each frequency band
            DE_delta = np.zeros(shape=[0], dtype=float)
            DE_theta = np.zeros(shape=[0], dtype=float)
            DE_alpha = np.zeros(shape=[0], dtype=float)
            DE_beta = np.zeros(shape=[0], dtype=float)
            DE_gamma = np.zeros(shape=[0], dtype=float)

            # Compute DE features for each frequency band in each segment
            for index in range(num_sample):
                DE_delta = np.append(DE_delta, compute_DE(delta[index * 100:(index + 1) * 100]))
                DE_theta = np.append(DE_theta, compute_DE(theta[index * 100:(index + 1) * 100]))
                DE_alpha = np.append(DE_alpha, compute_DE(alpha[index * 100:(index + 1) * 100]))
                DE_beta = np.append(DE_beta, compute_DE(beta[index * 100:(index + 1) * 100]))
                DE_gamma = np.append(DE_gamma, compute_DE(gamma[index * 100:(index + 1) * 100]))

            # Stack the DE features for each frequency band
            temp_de = np.vstack([temp_de, DE_delta])
            temp_de = np.vstack([temp_de, DE_theta])
            temp_de = np.vstack([temp_de, DE_alpha])
            temp_de = np.vstack([temp_de, DE_beta])
            temp_de = np.vstack([temp_de, DE_gamma])

        # Reshape the DE features to match the desired format
        temp_trial_de = temp_de.reshape(-1, 5, num_sample)
        print("temp_trial_de:", temp_trial_de.shape)  # Print the shape of the reshaped DE features
        temp_trial_de = temp_trial_de.transpose([2, 0, 1])  # Rearrange dimensions to match desired format
        decomposed_de = np.vstack([decomposed_de, temp_trial_de])  # Stack trial data

    print("trial_DE shape:", decomposed_de.shape)
    return decomposed_de, label


# Main script to extract features and save data
file_path = 'D:/BigData/SEED_IV/SEED_IV/eeg_raw_data/'

# List of participant names and short names used in file naming
people_name = ['1_20160518', '1_20161125','1_20161126',
               '2_20150915', '2_20150920','2_20151012',
               '3_20150919','3_20151018','3_20151101',
               '4_20151111', '4_20151118','4_20151123',
               '5_20160406', '5_20160413','5_20160420',
               '6_20150507','6_20150511','6_20150512',
               '7_20150715', '7_20150717','7_20150721',
               '8_20151103', '8_20151110','8_20151117',
               '9_20151028','9_20151119','9_20151209',
               '10_20151014', '10_20151021','10_20151023',
               '11_20150916', '11_20150921','11_20151011',
               '12_20150725','12_20150804','12_20150807',
               '13_20151115', '13_20151125','13_20161130',
               '14_20151205', '14_20151208','14_20151215',
               '15_20150508','15_20150514','15_20150527']

short_name = ['cz', 'cz','cz',
              'ha', 'ha', 'ha',
              'hql','hql','hql',
              'ldy','ldy','ldy',
              'ly','ly','ly',
              'mhw','mhw','mhw',
              'mz','mz','mz',
              'qyt','qyt','qyt',
              'rx','rx','rx',
              'tyc','tyc','tyc',
              'whh','whh','whh',
              'wll','wll','wll',
              'wq','wq','wq',
              'zjd','zjd','zjd',
              'zjy','zjy','zjy']

# Initialize empty arrays for storing the final DE features and labels
X = np.empty([0, 62, 5])
y = np.empty([0, 1])

# Loop through all participants to extract DE features
for i in range(len(people_name)):  # Loop through all 45 experiments (15 participants, 3 trials each)
    file_name = file_path + people_name[i]
    print('processing {}'.format(people_name[i]))
    decomposed_de, label = decompose(file_name, short_name[i])  # Extract DE features for each participant
    X = np.vstack([X, decomposed_de])  # Stack the features for all participants
    y = np.append(y, label)  # Stack the labels for all participants

# Save the extracted DE features and labels as .npy files
np.save("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/X_1D.npy", X)
np.save("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/y.npy", y)

# Load the saved features and labels
X = np.load('D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/X_1D.npy')
y = np.load('D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/y.npy')

def reshape(y,X):
    # Reshape the 62-channel data into an 8x9 matrix (based on the 10-20 electrode system)
    X89 = np.zeros((len(y), 8, 9, 5))  # Create an empty array for the 8x9x5 data
    X89[:, 0, 2, :] = X[:, 3, :]  # Assign values to specific positions in the 8x9 matrix
    X89[:, 0, 3:6, :] = X[:, 0:3, :]
    X89[:, 0, 6, :] = X[:, 4, :]

    # Assign values for the middle rows of the 8x9 matrix
    for i in range(5):
        X89[:, i + 1, :, :] = X[:, 5 + i * 9:5 + (i + 1) * 9, :]

    # Assign values for the last two rows of the 8x9 matrix
    X89[:, 6, 1:8, :] = X[:, 50:57, :]
    X89[:, 7, 2:7, :] = X[:, 57:62, :]
    return X89

X89 = reshape(y, X)

# Save the reshaped 8x9 matrix data
np.save("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/X89.npy", X89)


processing 1_20160518
1-336
temp_trial_de: (62, 5, 336)
2-190
temp_trial_de: (62, 5, 190)
3-398
temp_trial_de: (62, 5, 398)
4-260
temp_trial_de: (62, 5, 260)
5-176
temp_trial_de: (62, 5, 176)
6-324
temp_trial_de: (62, 5, 324)
7-306
temp_trial_de: (62, 5, 306)
8-418
temp_trial_de: (62, 5, 418)
9-290
temp_trial_de: (62, 5, 290)
10-338
temp_trial_de: (62, 5, 338)
11-100
temp_trial_de: (62, 5, 100)
12-220
temp_trial_de: (62, 5, 220)
13-434
temp_trial_de: (62, 5, 434)
14-338
temp_trial_de: (62, 5, 338)
15-518
temp_trial_de: (62, 5, 518)
16-282
temp_trial_de: (62, 5, 282)
17-136
temp_trial_de: (62, 5, 136)
18-358
temp_trial_de: (62, 5, 358)
19-280
temp_trial_de: (62, 5, 280)
20-96
temp_trial_de: (62, 5, 96)
21-224
temp_trial_de: (62, 5, 224)
22-224
temp_trial_de: (62, 5, 224)
23-350
temp_trial_de: (62, 5, 350)
24-274
temp_trial_de: (62, 5, 274)
trial_DE shape: (6870, 62, 5)
processing 1_20161125
1-442
temp_trial_de: (62, 5, 442)
2-202
temp_trial_de: (62, 5, 202)
3-278
temp_trial_de: (62, 5, 

## Loading and Reshaping Data

**Purpose**:  
Load a comprehensive EEG dataset from a single `.npy` file and organize it by participant for subsequent segmentation.

**Implementation**:  
- The code starts by loading `X89.npy`, a dataset of shape `(203970, 8, 9, 5)`.  
- This data is reshaped into `(15, 13598, 8, 9, 5)`, where the first dimension corresponds to the 15 participants.  
- By grouping the data in this manner, the EEG signals from each participant are kept together, facilitating participant-specific segment extraction.



## Defining Segment Length

**Segment Length (t)**:  
- Set to `6`, representing a 3-second segment since each time step is 0.5 seconds.  
- **Purpose**: Capturing temporal patterns over a 3-second window helps reveal meaningful temporal dependencies in the EEG data.



## Segmenting Data into Fixed-Length Sequences

**Function**: `segment_data(falx, t, lengths, labels)`  
**Purpose**:  
- Break down the participant-organized data into fixed-length sequences (`t=6`) and assign labels to each segment based on experimental conditions or trial definitions.

**Method**:  
1. **Calculate Boundaries**: Uses `np.cumsum(lengths)` to determine the segmentation points for different trials or conditions.  
2. **Iterative Extraction**: For each participant, the code iterates through the trials, extracting non-overlapping segments of length `t`.  
3. **Label Assignment**: Each extracted segment receives a label from the `labels` list, ensuring that the ground truth classification is maintained at a segment level.  
4. **Memory Management**: Pre-allocates a large array for efficiency, then resizes it after segmentation to exclude any unused space.


## Defining Trial Lengths and Labels

**Purpose**:  
- Provide essential metadata (segment lengths and labels) for accurately slicing the EEG data and linking each segment to its corresponding experimental label.

**Data**:  
- `lengths`: A list detailing the duration (in time steps) of each trial or experimental condition.  
- `all_label`: A list assigning a label to each trial, enabling the code to create a labeled dataset for classification tasks.



## Segmenting and Saving Data

**Process**:  
- Calls `segment_data` with the specified `lengths` and `all_label` lists to produce `new_x` and `new_y`:
  - `new_x` contains the segmented EEG data in 3D form `(participant, segment, t, 8, 9, 5)`.
  - `new_y` contains the corresponding labels for each segment.

**Verification**:  
- Prints the shapes of `new_x` and `new_y` to confirm that the segmentation and labeling steps were successful.

**Saving**:  
- Stores the final segmented data and labels into `.npy` files, for example:
  - `t6x_89.npy` for segmented data.
  - `t6y_89.npy` for labels.
  
This ensures that the segmented and labeled dataset is readily accessible for downstream analysis or machine learning model training.



In [3]:
# segements
import numpy as np

X89 = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/X89.npy")
print('{}-{}'.format('x_train shape', X89.shape))#x_train shape-(203970, 8, 9, 5)
img_rows, img_cols, num_chan = 8, 9, 5
falx = X89
falx = falx.reshape((15, int(X89.shape[0] / 15), img_rows, img_cols, num_chan)) #falx shape-(15, 13598, 8, 9, 5)
print('{}-{}'.format('falx shape', falx.shape))
t = 6 #(0.5s ->3s segement  6 pieces)

def segment_data(falx, t, lengths, labels):
  """Segments data into fixed-length segments with corresponding labels.

  Args:
    falx: The input data array.
    t: The length of each segment.
    lengths: A list of lengths for each segment.
    labels: A list of labels corresponding to each segment.

  Returns:
    new_x: The segmented data array.
    new_y: The corresponding label array.
  """

  # Calculate boundaries from lengths
  boundaries = np.cumsum(lengths)
  print('{}-{}'.format('boundaries', boundaries))

  total_segments = 100000 #pre-allocated number
  print('{}-{}'.format('total_segments', total_segments))

  # Pre-allocate new_x with correct dimensions
  new_x = np.empty([falx.shape[0], total_segments, t, 8, 9, 5])  
  new_y = np.array([])

  for nb in range(falx.shape[0]):
    z = 0
    i = 0
    for j, bound in enumerate(boundaries):
      while i + t <= bound:
        # Assign segments directly, taking all 6 time steps at once
        new_x[nb, z] = falx[nb, i:i + t]  # Assign to the first t indices of the segment
        new_y = np.append(new_y, labels[j])
        i = i + t
        z = z + 1
      i = bound
    print('{}-{}'.format(nb, z))
    # Resize new_x to exclude empty cells (segments that were not filled)
    new_x = new_x[:, :z, :, :, :, :]
  return new_x, new_y

lengths =[336,190,398,260,176,324,306,418,290,338,100,220,434,338,518,282,136,358,280,96,224,224,350,274,
          442, 202, 278,292,428,222,278, 368, 276, 166, 480, 100, 292, 216, 352, 122, 374, 392, 364, 86, 298, 352, 196, 152,
          340, 260, 184, 364, 386, 212, 516, 186, 208, 128, 414, 330, 314, 154, 230, 354, 114, 140, 366, 178, 318, 310, 330, 312]
all_label = [1,2,3,0,2,0,0,1,0,1,2,1,1,1,2,3,2,2,3,3,0,3,0,3,
             2,1,3,0,0,2,0,2,3,3,2,3,2,0,1,1,2,1,0,3,0,1,3,1,
             1,2,2,1,3,3,3,1,1,2,1,0,2,3,3,0,2,3,0,0,2,0,1,0]
new_x, new_y = segment_data(falx, 6, lengths, all_label)

print('{}-{}'.format('new_x shape', new_x.shape))
print('{}-{}'.format('new_y shape', new_y.shape))

np.save('D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/t'+str(t)+'x_89.npy', new_x)#new_x shape-(15, 2247, 6, 8, 9, 5)
np.save('D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/t'+str(t)+'y_89.npy', new_y)#new_y shape-(33705,)


x_train shape-(303690, 8, 9, 5)
falx shape-(15, 20246, 8, 9, 5)
boundaries-[  336   526   924  1184  1360  1684  1990  2408  2698  3036  3136  3356
  3790  4128  4646  4928  5064  5422  5702  5798  6022  6246  6596  6870
  7312  7514  7792  8084  8512  8734  9012  9380  9656  9822 10302 10402
 10694 10910 11262 11384 11758 12150 12514 12600 12898 13250 13446 13598
 13938 14198 14382 14746 15132 15344 15860 16046 16254 16382 16796 17126
 17440 17594 17824 18178 18292 18432 18798 18976 19294 19604 19934 20246]
total_segments-100000
0-3348
1-3348
2-3348
3-3348
4-3348
5-3348
6-3348
7-3348
8-3348
9-3348
10-3348
11-3348
12-3348
13-3348
14-3348
new_x shape-(15, 3348, 6, 8, 9, 5)
new_y shape-(50220,)


# Model Traning and Evaluation

## Objective: Train deep learning models for emotion recognition from EEG data 

## Data Handling:

Loaded and normalized EEG data.
Prepared data for input into the model by reshaping and selecting specific frequency bands.
## Model Design:

We Created three models seperately.
Used a softmax output layer for classification into four emotion classes.

## Training and Evaluation:

Used K-Fold cross-validation to ensure robustness.
Employed early stopping and learning rate reduction for efficient training.
Evaluated model performance on both test and validation sets.

## Results:

Calculated mean and standard deviation of accuracies.
Saved trained models for future use.

## Multi-CNN-LSTM

In [4]:
import numpy as np
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split, KFold

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Layer, Lambda
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM, BatchNormalization
from keras.models import Sequential, Model
from keras.utils import plot_model

import keras
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.config.list_physical_devices('GPU'))

from keras import backend as K
import time
import datetime
from keras.layers import Bidirectional
from keras.callbacks import EarlyStopping

#==================================Data Loading and Reshaping=====================================
num_classes = 4
batch_size = 64
img_rows, img_cols, num_chan = 8, 9, 4

falx = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/t6x_89.npy")
y = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/t6y_89.npy")
print('{}-{}'.format('falx shape', falx.shape))
print('{}-{}'.format('y shape', y.shape))

one_y = to_categorical(y, num_classes)  
print('{}-{}'.format('one_y categorical shape', one_y.shape))

# Reshape data into segments of length t=6 and select only 4 bands
one_falx_1 = falx.reshape((-1, 6, img_rows, img_cols, 5))  
one_falx = one_falx_1[:, :, :, :, 1:5]  # Extract only bands 1 to 4

#============================= Fixed 30% Test Set Before K-Fold ==============================
X_train_all, X_test, y_train_all, y_test = train_test_split(one_falx, one_y, 
                                                            test_size=0.3, 
                                                            random_state=42, 
                                                            stratify=one_y.argmax(1))

print("X_train_all shape:", X_train_all.shape)
print("y_train_all shape:", y_train_all.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Define base network
def create_base_network(input_dim):
    seq = Sequential()
    seq.add(Conv2D(64, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
    seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
    seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
    seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
    seq.add(MaxPooling2D(2, 2, name='pool1'))
    seq.add(Flatten(name='fla1'))
    seq.add(Dense(512, activation='relu', name='dense1'))
    seq.add(Reshape((1, 512), name='reshape'))
    return seq

acc_list = []
std_list = []
all_acc = []

# Initialize KFold on the training portion (70% of the data)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

K.clear_session()
start = time.time()

img_size = (img_rows, img_cols, num_chan)

for fold, (train_indices, val_indices) in enumerate(kf.split(X_train_all)):
    X_fold_train, X_fold_val = X_train_all[train_indices], X_train_all[val_indices]
    y_fold_train, y_fold_val = y_train_all[train_indices], y_train_all[val_indices]

    base_network = create_base_network(img_size)
    input_1 = Input(shape=img_size)
    input_2 = Input(shape=img_size)
    input_3 = Input(shape=img_size)
    input_4 = Input(shape=img_size)
    input_5 = Input(shape=img_size)
    input_6 = Input(shape=img_size)

    out_all = Concatenate(axis=1)([
        base_network(input_1), base_network(input_2), base_network(input_3), 
        base_network(input_4), base_network(input_5), base_network(input_6)
    ])
    lstm_layer = LSTM(128, name='lstm')(out_all)
    out_layer = Dense(4, activation='softmax', name='out')(lstm_layer)
    model = Model([input_1, input_2, input_3, input_4, input_5, input_6], out_layer)

    # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # TensorBoard
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Train the model on this fold's train/val split
    model.fit([
       X_fold_train[:, 0, :, :, :], 
       X_fold_train[:, 1, :, :, :], 
       X_fold_train[:, 2, :, :, :], 
       X_fold_train[:, 3, :, :, :], 
       X_fold_train[:, 4, :, :, :], 
       X_fold_train[:, 5, :, :, :]], 
      y_fold_train, epochs=100, batch_size=128, verbose=1,
      validation_data=([
         X_fold_val[:, 0, :, :, :],
         X_fold_val[:, 1, :, :, :],
         X_fold_val[:, 2, :, :, :],
         X_fold_val[:, 3, :, :, :],
         X_fold_val[:, 4, :, :, :],
         X_fold_val[:, 5, :, :, :]],
         y_fold_val),
      callbacks=[early_stopping, tensorboard_callback])

    # Evaluate on the fixed 30% test set
    scores = model.evaluate([
       X_test[:, 0, :, :, :], 
       X_test[:, 1, :, :, :], 
       X_test[:, 2, :, :, :], 
       X_test[:, 3, :, :, :], 
       X_test[:, 4, :, :, :], 
       X_test[:, 5, :, :, :]], 
      y_test, verbose=0)

    model.save(f'Multi-CNN-LSTM_fold{fold+1}.h5')
    print(f"Fold {fold+1} Test Accuracy: {scores[1] * 100:.2f}%")
    all_acc.append(scores[1] * 100)

print("all_Vali:", all_acc)
print("Mean Vali:", np.mean(all_acc))
print("Std Vali:", np.std(all_acc))

end = time.time()
print("Run time: %.2f seconds" % (end - start))



[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
falx shape-(15, 3348, 6, 8, 9, 5)
y shape-(50220,)
one_y categorical shape-(50220, 4)
X_train_all shape: (35154, 6, 8, 9, 4)
y_train_all shape: (35154, 4)
X_test shape: (15066, 6, 8, 9, 4)
y_test shape: (15066, 4)
Epoch 1/100
220/220 [==============================] - 14s 30ms/step - loss: 1.3740 - accuracy: 0.2895 - val_loss: 1.3801 - val_accuracy: 0.3085
Epoch 2/100
220/220 [==============================] - 6s 25ms/step - loss: 1.3398 - accuracy: 0.3302 - val_loss: 1.3193 - val_accuracy: 0.3446
Epoch 3/100
220/220 [==============================] - 6s 25ms/step - loss: 1.2782 - accuracy: 0.3866 - val_loss: 1.2611 - val_accuracy: 0.3795
Epoch 4/100
220/220 [==============================] - 6s 25ms/step - loss: 1.2051 - accuracy: 0.4388 - val_loss: 1.1489 - val_accuracy: 0.4777
Epoch 5/100
220/220 [==============================] - 5s 25ms/step - loss: 1.1065 - accuracy: 0.5064 - val_loss: 1.0692 - val_accuracy: 0.529

### output
- all_Vali: [76.105135679245, 74.85065460205078, 75.9922981262207, 75.7732629776001, 73.70237708091736]
- Mean Vali: 75.28474569320679
- Std Vali: 0.9060513023429172

## Attention_BiLSTM-CNN

In [5]:
import numpy as np
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.model_selection import train_test_split, KFold

from keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Layer, Lambda
from keras.layers import Flatten, Dense, Concatenate, Reshape, LSTM, BatchNormalization
from keras.models import Sequential, Model
from keras.utils import plot_model
from keras.layers import Bidirectional
from keras.callbacks import EarlyStopping
import keras
import os
import tensorflow as tf
import datetime
from keras import backend as K

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.config.list_physical_devices('GPU'))

#==================================Data Loading and reshaping=====================================
num_classes = 4
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

falx = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/t6x_89.npy")
y = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/t6y_89.npy")
print('{}-{}'.format('falx shape', falx.shape))  # e.g. (N, 6, 8, 9, 5)
print('{}-{}'.format('y shape', y.shape))

one_y = to_categorical(y, num_classes)  
print('{}-{}'.format('one_y categorical shape', one_y.shape))

one_falx_1 = falx.reshape((-1, 6, img_rows, img_cols, 5))  # reshape into segments of length t=6
one_falx = one_falx_1[:, :, :, :, 1:5]  # only 4 bands (excluding the first band if it's a "sleep" feature)

#==================================30% Test Split Before K-Fold====================================

X_train_all, X_test, y_train_all, y_test = train_test_split(one_falx, one_y, 
                                                            test_size=0.3, 
                                                            random_state=42,
                                                            stratify=one_y.argmax(1))

print("X_train_all shape:", X_train_all.shape)
print("X_test shape:", X_test.shape)
print("y_train_all shape:", y_train_all.shape)
print("y_test shape:", y_test.shape)

# After this split:
# - X_test and y_test remain fixed for final evaluation (30% of entire data)
# - X_train_all and y_train_all (70%) are used for K-Fold

# Define the AttentionLayer
class AttentionLayer(Layer):
    def __init__(self, **kwargs):
        super(AttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                 initializer='glorot_uniform',
                                 trainable=True)
        self.b = self.add_weight(shape=(input_shape[-1],),
                                 initializer='zero', trainable=True)
        super(AttentionLayer, self).build(input_shape)

    def call(self, x):
        # x shape: (None, 6, 256)
        e = K.tanh(K.dot(x, self.W) + self.b)  # shape: (None, 6, 256)
        
        # Apply attention mechanism (just an example, ensure correctness)
        e_sum = K.sum(e, axis=-1)  # shape: (None, 6)
        a = K.softmax(e_sum)       # shape: (None, 6)
        
        # Weighted sum of the inputs by attention weights
        a_expanded = K.expand_dims(a, axis=-1)  # shape: (None, 6, 1)
        output = x * a_expanded
        return K.sum(output, axis=1)  # shape: (None, 256)

    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[-1])

#def base-network
def create_base_network(input_dim):
    seq = Sequential()
    seq.add(Conv2D(32, 5, activation='relu', padding='same', name='conv1', input_shape=input_dim))
    seq.add(BatchNormalization())
    seq.add(Dropout(0.2))
    seq.add(Conv2D(128, 4, activation='relu', padding='same', name='conv2'))
    seq.add(BatchNormalization())
    seq.add(Dropout(0.2))
    seq.add(Conv2D(256, 4, activation='relu', padding='same', name='conv3'))
    seq.add(BatchNormalization())
    seq.add(Dropout(0.2))
    seq.add(Conv2D(64, 1, activation='relu', padding='same', name='conv4'))
    seq.add(MaxPooling2D(2, 2, name='pool1'))
    seq.add(Flatten(name='fla1'))
    seq.add(Dense(256, activation='relu', name='dense1'))
    seq.add(BatchNormalization())
    seq.add(Dropout(0.3))
    seq.add(Reshape((1, 256), name='reshape'))
    return seq

#====================================Training with K-Fold on the 70% training set=================================

acc_list = []
std_list = []
all_acc = []

# Initialize KFold with the desired number of splits
kf = KFold(n_splits=5, shuffle=True, random_state=42)

K.clear_session()
start = datetime.datetime.now()

img_size = (img_rows, img_cols, num_chan)

for fold, (train_indices, val_indices) in enumerate(kf.split(X_train_all)):
    X_fold_train, X_fold_val = X_train_all[train_indices], X_train_all[val_indices]
    y_fold_train, y_fold_val = y_train_all[train_indices], y_train_all[val_indices]

    # Create the model
    base_network = create_base_network(img_size)
    input_1 = Input(shape=img_size)
    input_2 = Input(shape=img_size)
    input_3 = Input(shape=img_size)
    input_4 = Input(shape=img_size)
    input_5 = Input(shape=img_size)
    input_6 = Input(shape=img_size)

    out_all = Concatenate(axis=1)([
        base_network(input_1), 
        base_network(input_2), 
        base_network(input_3), 
        base_network(input_4), 
        base_network(input_5), 
        base_network(input_6)
    ])
    # Apply attention
    attention_output = AttentionLayer()(out_all)
    lstm_layer = Bidirectional(LSTM(128, name='lstm'))(out_all)
    out_layer = Dense(4, activation='softmax', name='out')(lstm_layer)
    model = Model([input_1, input_2, input_3, input_4, input_5, input_6], out_layer)

    # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate=0.0005),
                  metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # TensorBoard
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Fit the model on the fold's train/val splits
    model.fit([
        X_fold_train[:, 0, :, :, :], 
        X_fold_train[:, 1, :, :, :], 
        X_fold_train[:, 2, :, :, :], 
        X_fold_train[:, 3, :, :, :], 
        X_fold_train[:, 4, :, :, :], 
        X_fold_train[:, 5, :, :, :]], 
        y_fold_train, epochs=150, batch_size=batch_size, verbose=1,
        validation_data=([
            X_fold_val[:, 0, :, :, :],
            X_fold_val[:, 1, :, :, :],
            X_fold_val[:, 2, :, :, :],
            X_fold_val[:, 3, :, :, :],
            X_fold_val[:, 4, :, :, :],
            X_fold_val[:, 5, :, :, :]],
            y_fold_val),
        callbacks=[early_stopping, tensorboard_callback])

    # Evaluate on the fixed 30% test set
    scores = model.evaluate([
        X_test[:, 0, :, :, :], 
        X_test[:, 1, :, :, :], 
        X_test[:, 2, :, :, :], 
        X_test[:, 3, :, :, :], 
        X_test[:, 4, :, :, :], 
        X_test[:, 5, :, :, :]], 
        y_test, verbose=0)
    
    model.save(f'Attention-BiLSTM-CNN_fold{fold+1}.h5')
    print(f"Fold {fold+1} Test Accuracy: {scores[1]*100:.2f}%")
    all_acc.append(scores[1] * 100)

print("all_Vali:", all_acc)
print("Mean Vali:", np.mean(all_acc))
print("Std Vali:", np.std(all_acc))

end = datetime.datetime.now()
run_time = end - start
print("Run time:", run_time)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
falx shape-(15, 3348, 6, 8, 9, 5)
y shape-(50220,)
one_y categorical shape-(50220, 4)
X_train_all shape: (35154, 6, 8, 9, 4)
X_test shape: (15066, 6, 8, 9, 4)
y_train_all shape: (35154, 4)
y_test shape: (15066, 4)
Epoch 1/150
220/220 [==============================] - 14s 39ms/step - loss: 1.2672 - accuracy: 0.4083 - val_loss: 1.2895 - val_accuracy: 0.4271
Epoch 2/150
220/220 [==============================] - 7s 33ms/step - loss: 1.1160 - accuracy: 0.5113 - val_loss: 1.5310 - val_accuracy: 0.4062
Epoch 3/150
220/220 [==============================] - 7s 33ms/step - loss: 1.0061 - accuracy: 0.5738 - val_loss: 1.0289 - val_accuracy: 0.5649
Epoch 4/150
220/220 [==============================] - 7s 33ms/step - loss: 0.9155 - accuracy: 0.6166 - val_loss: 0.8955 - val_accuracy: 0.6225
Epoch 5/150
220/220 [==============================] - 7s 33ms/step - loss: 0.8430 - accuracy: 0.6554 - val_loss: 1.0179 - val_accuracy: 0.589

### output
- all_Vali: [81.64077997207642, 78.30213904380798, 82.49037861824036, 81.24253153800964, 81.8067193031311]
- Mean Vali: 81.0965096950531
- Std Vali: 1.4541889912968695

## ResCNN-TransGRU

In [6]:
import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.layers import (Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Concatenate, 
                          Reshape, BatchNormalization, Bidirectional, GRU, Add, MultiHeadAttention, 
                          LayerNormalization, GlobalAveragePooling1D)
from keras.models import Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
import keras.backend as K
import tensorflow as tf
from sklearn.model_selection import KFold
import datetime
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
print(tf.config.list_physical_devices('GPU'))

# Data Loading and Reshaping
num_classes = 4
batch_size = 128
img_rows, img_cols, num_chan = 8, 9, 4

falx = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/t6x_89.npy")
y = np.load("D:/BigData/SEED_IV/SEED_IV/DE0.5s/session_1_2_3/t6y_89.npy")
print('{}-{}'.format('falx shape', falx.shape))
print('{}-{}'.format('y shape', y.shape))

one_y = to_categorical(y, num_classes)
print('{}-{}'.format('one_y categorical shape', one_y.shape))

one_falx_1 = falx.reshape((-1, 6, img_rows, img_cols, 5))  # reshape each person's segments
one_falx = one_falx_1[:, :, :, :, 1:5]  # only 4 bands

#=========================== Fixed 30% Test Set Before K-Fold ===========================
X_train_all, X_test, y_train_all, y_test = train_test_split(one_falx, one_y,
                                                            test_size=0.3,
                                                            random_state=42,
                                                            stratify=one_y.argmax(1))
print("X_train_all shape:", X_train_all.shape)
print("y_train_all shape:", y_train_all.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

# Base CNN Network with Residual Blocks
def create_base_network(input_dim):
    input_layer = Input(shape=input_dim)
    x = Conv2D(32, 5, activation='relu', padding='same', name='conv1')(input_layer)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    
    # First Residual Block
    residual = Conv2D(128, 4, activation='relu', padding='same', name='conv2')(x)
    residual = BatchNormalization()(residual)
    residual = Dropout(0.2)(residual)
    x = Conv2D(128, 4, activation='relu', padding='same', name='conv2_residual')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Add()([x, residual])
    
    # Second Residual Block
    residual = Conv2D(256, 4, activation='relu', padding='same', name='conv3')(x)
    residual = BatchNormalization()(residual)
    residual = Dropout(0.2)(residual)
    x = Conv2D(256, 4, activation='relu', padding='same', name='conv3_residual')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    x = Add()([x, residual])
    
    x = Conv2D(64, 1, activation='relu', padding='same', name='conv4')(x)
    x = MaxPooling2D(2, 2, name='pool1')(x)
    x = Flatten(name='fla1')(x)
    x = Dense(256, activation='relu', name='dense1')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.3)(x)
    x = Reshape((1, 256), name='reshape')(x)
    
    return Model(inputs=input_layer, outputs=x)

# Vision Transformer Encoder Block
def transformer_encoder(inputs, embed_dim, num_heads, ff_dim, rate=0.1):
    # Multi-Head Self Attention
    attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)(inputs, inputs)
    attn_output = Dropout(rate)(attn_output)
    # Add & Norm
    out1 = LayerNormalization(epsilon=1e-6)(inputs + attn_output)
    # Feed Forward Network
    ffn_output = Dense(ff_dim, activation='relu')(out1)
    ffn_output = Dense(embed_dim)(ffn_output)
    ffn_output = Dropout(rate)(ffn_output)
    # Add & Norm
    return LayerNormalization(epsilon=1e-6)(out1 + ffn_output)

acc_list = []
std_list = []
all_acc = []

K.clear_session()
start = time.time()

img_size = (img_rows, img_cols, num_chan)

# K-Fold on the 70% training data
kf = KFold(n_splits=5, shuffle=True, random_state=42)

for fold, (train_indices, val_indices) in enumerate(kf.split(X_train_all)):
    X_fold_train, X_fold_val = X_train_all[train_indices], X_train_all[val_indices]
    y_fold_train, y_fold_val = y_train_all[train_indices], y_train_all[val_indices]

    # Create base network
    base_network = create_base_network(img_size)
    
    inputs = [Input(shape=img_size) for _ in range(6)]
    out_all = Concatenate(axis=1)([base_network(inp) for inp in inputs])

    # Transformer Encoder Block
    embed_dim = 256
    num_heads = 4
    ff_dim = 512
    transformer_output = transformer_encoder(out_all, embed_dim, num_heads, ff_dim)

    # Bidirectional GRU Layer
    bidir_gru = Bidirectional(GRU(128, return_sequences=False))(transformer_output)

    # Output Layer
    out_layer = Dense(4, activation='softmax', name='out')(bidir_gru)
    model = Model(inputs, out_layer)

    # TensorBoard
    log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    # Compile model
    model.compile(loss=keras.losses.categorical_crossentropy,
                  optimizer=keras.optimizers.Adam(learning_rate=0.0005),
                  metrics=['accuracy'])
    
    # Early stopping and learning rate reduction
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-6, verbose=1)

    # Train the model on this fold's train/val splits
    model.fit(
        [X_fold_train[:, i, :, :, :] for i in range(6)],
        y_fold_train,
        epochs=150,
        batch_size=batch_size,
        verbose=1,
        validation_data=([X_fold_val[:, i, :, :, :] for i in range(6)], y_fold_val),
        callbacks=[early_stopping, reduce_lr, tensorboard_callback]
    )

    # Evaluate the model on the fixed 30% test set
    scores = model.evaluate(
        [X_test[:, i, :, :, :] for i in range(6)],
        y_test,
        verbose=0
    )
    model.save(f'ResCNN-TransGRU_fold{fold+1}.h5')
    
    print(f"Fold {fold+1} Test Accuracy: {scores[1] * 100:.2f}%")
    all_acc.append(scores[1] * 100)

print("all_Vali:", all_acc)
print("Mean acc:", np.mean(all_acc))
print("Std acc:", np.std(all_acc))
end = time.time()
print("Run time: %.2f seconds" % (end - start))


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
falx shape-(15, 3348, 6, 8, 9, 5)
y shape-(50220,)
one_y categorical shape-(50220, 4)
X_train_all shape: (35154, 6, 8, 9, 4)
y_train_all shape: (35154, 4)
X_test shape: (15066, 6, 8, 9, 4)
y_test shape: (15066, 4)
Epoch 1/150
220/220 [==============================] - 20s 63ms/step - loss: 1.3023 - accuracy: 0.3950 - val_loss: 1.2863 - val_accuracy: 0.4416 - lr: 5.0000e-04
Epoch 2/150
220/220 [==============================] - 12s 56ms/step - loss: 1.1473 - accuracy: 0.4917 - val_loss: 1.1225 - val_accuracy: 0.5230 - lr: 5.0000e-04
Epoch 3/150
220/220 [==============================] - 12s 57ms/step - loss: 1.0459 - accuracy: 0.5530 - val_loss: 1.1641 - val_accuracy: 0.5053 - lr: 5.0000e-04
Epoch 4/150
220/220 [==============================] - 12s 57ms/step - loss: 0.9756 - accuracy: 0.5843 - val_loss: 0.9687 - val_accuracy: 0.5867 - lr: 5.0000e-04
Epoch 5/150
220/220 [==============================] - 12s 56ms/step - 

### ouput
- all_Vali: [85.43741106987, 85.74272990226746, 85.70954203605652, 85.19182205200195, 85.37766933441162]
- Mean acc: 85.49183487892151
- Std acc: 0.20801352738163414